In [0]:
%pip install -U -qqqq mlflow databricks-openai databricks-agents
dbutils.library.restartPython()

In [0]:
dbutils.widgets.text("catalog_name", "btafur_catalog")
dbutils.widgets.text("schema_name", "default")
dbutils.widgets.text("model_name", "quickstart_agent")

catalog_name = dbutils.widgets.get("catalog_name")
schema_name = dbutils.widgets.get("schema_name")
model_name = dbutils.widgets.get("model_name")
dbutils.widgets.text("experiment_name", f"/Users/{dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()}/{model_name}_{catalog_name}")
experiment_name = dbutils.widgets.get("experiment_name")

import mlflow

mlflow.set_experiment(experiment_name)

registered_model_name = f"{catalog_name}.{schema_name}.{model_name}"

In [0]:
#https://docs.databricks.com/aws/en/generative-ai/agent-framework/deploy-agent#deploy-an-agent-using-deploy

from mlflow import MlflowClient
from databricks.agents import get_deployments, delete_deployment
from databricks import agents

client = MlflowClient()

registered_model_version = client.get_model_version_by_alias(registered_model_name, "Champion")

deployments = get_deployments(model_name=registered_model_name)

latest_version = registered_model_version.version

for deployment in deployments:
    if deployment.model_name == registered_model_name:
        print(f"Deleting deployment: model={deployment.model_name}, version={deployment.model_version}")
        delete_deployment(model_name=deployment.model_name, model_version=deployment.model_version)


deployment_info = agents.deploy(
    model_name=registered_model_name,
    model_version=registered_model_version.version,
)